# quant-econ Solutions: Default Risk and Income Fluctuations

Solutions for http://quant-econ.net/jl/arellano.html

In [7]:
using QuantEcon, QuantEcon.Models
using PyPlot

ErrorException("Failed to pyimport("matplotlib"): PyPlot will not work until you have a functioning matplotlib module.  PyError (:PyImport_ImportModule) <type 'exceptions.ImportError'>
ImportError('cannot import name scimath',)
  File "/home/matthewmckay/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py", line 180, in <module>
    from matplotlib.cbook import is_string_like
  File "/home/matthewmckay/anaconda/lib/python2.7/site-packages/matplotlib/cbook.py", line 33, in <module>
    import numpy as np
  File "/home/matthewmckay/anaconda/lib/python2.7/site-packages/numpy/__init__.py", line 170, in <module>
    from . import add_newdocs
  File "/home/matthewmckay/anaconda/lib/python2.7/site-packages/numpy/add_newdocs.py", line 13, in <module>
    from numpy.lib import add_newdoc
  File "/home/matthewmckay/anaconda/lib/python2.7/site-packages/numpy/lib/__init__.py", line 17, in <module>
    from . import scimath as emath
")


Compute the value function, policy and equilibrium prices

In [4]:
ae = ArellanoEconomy(   β=.953,     # time discount rate
                        γ=2.,       # risk aversion
                        r=0.017,    # international interest rate
                        ρ=.945,     # persistence in output 
                        η=0.025,    # st dev of output shock
                        θ=0.282,    # prob of regaining access 
                        ny=21,      # number of points in y grid
                        nB=251)     # number of points in B grid

ArellanoEconomy(0.953,2.0,0.017,0.945,0.025,0.282,21,251,[0.795083,0.813526,0.832396,0.851704,0.87146,0.891674,0.912357,0.93352,0.955174,0.97733  …  1.0232,1.04693,1.07121,1.09606,1.12149,1.1475,1.17412,1.20135,1.22922,1.25773],[0.795083,0.813526,0.832396,0.851704,0.87146,0.891674,0.912357,0.93352,0.955174,0.97733  …  0.978368,0.978368,0.978368,0.978368,0.978368,0.978368,0.978368,0.978368,0.978368,0.978368],[-0.4,-0.3968,-0.3936,-0.3904,-0.3872,-0.384,-0.3808,-0.3776,-0.3744,-0.3712  …  0.3712,0.3744,0.3776,0.3808,0.384,0.3872,0.3904,0.3936,0.3968,0.4],21x21 Array{Float64,2}:
 0.48171      0.326514     0.154936     …  0.0          0.0        
 0.180714     0.321116     0.319859        0.0          0.0        
 0.0375843    0.156704     0.327656        0.0          0.0        
 0.00406984   0.037836     0.166561        0.0          0.0        
 0.000221534  0.0044974    0.0419021       0.0          0.0        
 5.93963e-6   0.000261594  0.00519128   …  0.0          0.0        
 7.74751e

Compute the bond price schedule as seen in figure 3 of Arellano (2008)

In [5]:
#-!-EDIT-!-#
# Create "Y High" and "Y Low" values as 5% devs from mean
high, low = np.mean(ae.ygrid)*1.05, np.mean(ae.ygrid)*.95
iy_high, iy_low = (np.searchsorted(ae.ygrid, x) for x in (high, low))

fig, ax = plt.subplots(figsize=(10, 6.5))
ax.set_title("Bond price schedule $q(y, B')$")

# Extract a suitable plot grid
x = []
q_low = []
q_high = []
for i:ae.nB
    b = ae.Bgrid[i]
    if -0.35 <= b <= 0:  # To match fig 3 of Arellano
        x.append(b)
        q_low.append(ae.Q[iy_low, i])
        q_high.append(ae.Q[iy_high, i])
    end
end
ax.plot(x, q_high, label=r"$y_H$", lw=2, alpha=0.7)
ax.plot(x, q_low, label=r"$y_L$", lw=2, alpha=0.7)
ax.set_xlabel(r"$B'$")
ax.legend(loc='upper left', frameon=False)
plt.show()

LoadError: np not defined
while loading In[5], in expression starting on line 3

Draw a plot of the value functions

In [ ]:
#-!-EDIT-!-#
# Create "Y High" and "Y Low" values as 5% devs from mean
high, low = np.mean(ae.ygrid)*1.05, np.mean(ae.ygrid)*.95
iy_high, iy_low = (np.searchsorted(ae.ygrid, x) for x in (high, low))

fig, ax = plt.subplots(figsize=(10, 6.5))
ax.set_title("Value Functions")
ax.plot(ae.Bgrid, ae.V[iy_high], label=r"$y_H$", lw=2, alpha=0.7)
ax.plot(ae.Bgrid, ae.V[iy_low], label=r"$y_L$", lw=2, alpha=0.7)
ax.legend(loc='upper left')
ax.set_xlabel(r"$B$")
ax.set_ylabel(r"$V(y, B)$")
ax.set_xlim(ae.Bgrid.min(), ae.Bgrid.max())
plt.show()

Draw a heat map for default probability

In [ ]:
#-!-EDIT-!-#
xx, yy = ae.Bgrid, ae.ygrid
zz = ae.default_prob

# Create figure
fig, ax = plt.subplots(figsize=(10, 6.5))
fig.suptitle("Probability of Default")
hm = ax.pcolormesh(xx, yy, zz)
cax = fig.add_axes([.92, .1, .02, .8])
fig.colorbar(hm, cax=cax)
ax.axis([xx.min(), 0.05, yy.min(), yy.max()])
ax.set_xlabel(r"$B'$")
ax.set_ylabel(r"$y$")
plt.show()

Plot a time series of major variables simulated from the model.

In [ ]:
#-!-EDIT-!-#
T = 250
y_vec, B_vec, q_vec, default_vec = ae.simulate(T)

# Pick up default start and end dates
start_end_pairs = []
i = 0
while i < len(default_vec):
    if default_vec[i] == 0:
        i += 1
    else:
        # If we get to here we're in default
        start_default = i
        while i < len(default_vec) and default_vec[i] == 1:
            i += 1
        end
        end_default = i - 1
        start_end_pairs.append((start_default, end_default))
    end
end
plot_series = y_vec, B_vec, q_vec
titles = 'output', 'foreign assets', 'bond price'

fig, axes = plt.subplots(len(plot_series), 1, figsize=(10, 12))
p_args = {'lw': 2, 'alpha': 0.7}
fig.subplots_adjust(hspace=0.3)

#-!-EDIT-!-#
for ax, series, title in zip(axes, plot_series, titles):
    # determine suitable y limits
    s_max, s_min = max(series), min(series)
    s_range = s_max - s_min
    y_max = s_max + s_range * 0.1
    y_min = s_min - s_range * 0.1
    ax.set_ylim(y_min, y_max)
    for pair in start_end_pairs:
        ax.fill_between(pair, (y_min, y_min), (y_max, y_max), color='k', alpha=0.3)
        
    ax.grid()
    ax.set_title(title)
    ax.plot(range(T), series, **p_args)
    ax.set_xlabel(r"time")

plt.show()